# Final Project Starting Guide

Hello everyone, welcome to the final project! This notebook is provided to you to reiterate the rules and guidelines, and give you some starting points.

### What we provide

In this project, we will provide you with 
- This starting guide
- A working API that you can access under ASU network (i.e., on campus or with VPN)
- A starting development data that you can use to develop your agent. It contains 1,000 instances with {domain, input, expected_output}

### Your goal

In this project, you will implement an inference-time agent to solve reasoning requests, as those provided in the development data. The grading of this project will be effort-based and you will get full credit if you produce the minimum deliverables below, with subject to the rules and requirements below.

#### Minimum Deliverables

1. A working agent loop (in the form of a Github project) that the TA can run, and implements *at least three* inference-time algorithms or techniques.
2. Outputs from your agent on the released test data (see important dates). 
3. A short one-page report on how your agent works, and pointer to important techniques (referece to code blocks).

#### Rules and Requirements
1. You must only use our provided API call to access LLMs; meaning that you cannot use any other LLMs in any other way within your agent loop. Some exceptions may be made if you call certain external tools (e.g., Google search) that use some LLMs internally. Please discuss any exceptions with us to avoid penalties up to 50% of the project grade.
2. You must not hardcode a full delegation to an external tool (e.g., google_search(input_problem)). Such delegations must be automatically selected/decided by your agent. Hardcode delegations will lead to a zero.
3. You cannot use Cursor or any AI coding aids to implement the final project. You can, however, ask LLMs (or other online resources) for conceptual clarification or code examples. Your final project should not contain any blocks of code (i.e., > 3 lines) that are written by AI. Violations will lead to a zero.
4. Your agent should be able to run efficiently, with <20 LLM calls per question. Exceptions may be made when you have a complicated agent but please discuss with us. Up to 10% of the project grade may be deducted if we observe very inefficient LLM usages that do not clearly benefit the performance.
5. Your agent must run without any requests to any paid services (paid is defined by if the TA has to pay to run it, regardless of whether you actuallly pay for it or not.) Violations will lead to a zero.
6. You must submit a Github project link as your code submission. All changes must be tracked and any commits should be within 100 lines of +/- with good messages. Points will be deducted to up to 25% of the project grade if we observe "magic commits" or too few commits. 


### Suggestions
1. Start early, please.
2. You should consider how you can evaluate whether your output is good enough compared to the provided expected_outputs, and we will not release how we will actually evaluate your outputs; meaning that you have to try to predict how we will evaluate things.
3. Start with a basic implementation, and iterate based on mistakes/feedbacks.
4. Find more development data, or create your own cases to stree-test your agent. 
5. You are free to modify any provided code in this starting guide, or not using any of these code at all.

### Important dates
- **Release of final test data**: 11/25/2025
- **Deadline for submitting all deliverables**: 12/05/2025

### Extra Credit. 
The top 20 projects (ranked by performance metrics on the test data and at the TA's discretion of implementation quality) will be given extra credits. The actual credits will be between 1% to 7.5% depending on the ranking.

In [1]:
# %% Minimal setup
# If needed (uncomment in a notebook):
# !pip install requests python-dotenv

import os, json, textwrap, re, time
import requests

API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")  
MODEL    = os.getenv("MODEL_NAME", "bens_model")              

def call_model_chat_completions(prompt: str,
                                system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
                                model: str = MODEL,
                                temperature: float = 0.0,
                                timeout: int = 60) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": 2048,
    }

    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = resp.status_code
        hdrs   = dict(resp.headers)
        if status == 200:
            data = resp.json()
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = resp.json()
            except Exception:
                err_text = resp.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}


## 1) Smoke test: direct inference

We’ll do a single request with a strict instruction to answer briefly.  
*If you see an auth error, set `OPENAI_API_KEY` and (if needed) `API_BASE`/`MODEL_NAME`.*


In [4]:
# %% Direct call example
demo_prompt = "What is 17 + 28? Answer with just the number."
result = call_model_chat_completions(demo_prompt)
print("OK:", result["ok"], "HTTP:", result["status"])
print("MODEL SAYS:", (result["text"] or "").strip())

# Optional: Inspect rate-limit headers if your provider exposes them
for k in ["x-ratelimit-remaining-requests", "x-ratelimit-limit-requests", "x-request-id"]:
    if k in result["headers"]:
        print(f"{k}: {result['headers'][k]}")


OK: True HTTP: 200
MODEL SAYS: 45


## 2) A tiny test set (3 questions)

We’ll cover:
1. **Math reasoning** — inequality solving,
2. **Common sense** — buoyancy/ice & water,
3. **Logic** — a classic race-position puzzle.

We also tightly constrain the required answer forms to enable simple auto‑grading.


In [ ]:
# %% Define three tests: input + expected
tests = [
    {
        "id": "math_inequality",
        "type": "numeric",  # grader will prefer numeric extraction
        "prompt": "Let $ABCD$ be a convex quadrilateral with $AB = CD = 10$ , $BC = 14$ , and $AD = 2\\sqrt{65}$ . Assume that the diagonals of $ABCD$ intersect at point $P$ , and that the sum of the areas of triangles $APB$ and $CPD$ equals the sum of the areas of triangles $BPC$ and $APD$ . Find the area of quadrilateral $ABCD$ .",
        "expected": "112",    
    },
    {
        "id": "commonsense_ice",
        "type": "text",
        "prompt": (
            "You place an ice cube in a glass of water and mark the water level. "
            "After the ice melts, does the water level rise, fall, or stay the same? "
            "Answer with exactly one of: 'rise', 'fall', 'stay the same'."
        ),
        "expected": "stay the same",
    },
    {
        "id": "logic_race",
        "type": "text",
        "prompt": (
            "In a race, you pass the person in second place. What position are you now in? "
            "Answer with a single word like 'first', 'second', 'third'."
        ),
        "expected": "second",
    },
]


## 3) Minimal evaluator

We provide some example code to decide whether the agent outputs match the expected outputs, just to give you an idea of how evaluations can be done. You are free to use this code, or not.

In [4]:
# %% Simple normalization and evaluation helpers
def normalize_text(s: str) -> str:
    s = (s or "").strip().lower()
    # Remove surrounding punctuation and extra whitespace
    s = re.sub(r"[^\w\s\-']", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    # Map common synonyms used in these tests
    synonyms = {
        "unchanged": "stay the same",
        "no change": "stay the same",
        "same": "stay the same",
        "second place": "second",
        "2nd": "second",
        "first place": "first",
        "third place": "third",
    }
    return synonyms.get(s, s)

def extract_number(s: str):
    # Returns first number occurrence as string if found, else None
    if not s:
        return None
    m = re.search(r"[-+]?\d+(\.\d+)?", s)
    return m.group(0) if m else None

def grade(expected: str, got: str, kind: str) -> bool:
    if kind == "numeric":
        exp_num = extract_number(expected)
        got_num = extract_number(got)
        return (exp_num is not None) and (got_num == exp_num)
    else:
        return normalize_text(got) == normalize_text(expected)

def evaluate_tests(tests, model=MODEL):
    rows = []
    for t in tests:
        r = call_model_chat_completions(
            t["prompt"],
            system="You are a careful solver. Reply ONLY with the final answer, nothing else.",
            model=model,
            temperature=0.0,
        )
        got = (r["text"] or "").strip()
        is_correct = grade(t["expected"], got, t["type"])
        rows.append({
            "id": t["id"],
            "expected": t["expected"],
            "got": got,
            "correct": is_correct,
            "status": r["status"],
            "error": r["error"],
        })
        # Tiny pacing to be polite to the API
        time.sleep(0.2)

    # Print a small report
    correct = sum(1 for x in rows if x["correct"])
    print(f"Score: {correct}/{len(rows)} correct")
    for x in rows:
        mark = "✅" if x["correct"] else "❌"
        print(f"{mark} {x['id']}: expected={x['expected']!r}, got={x['got']!r} (HTTP {x['status']})")
        if x["error"]:
            print("   error:", x["error"])
    return rows

results = evaluate_tests(tests)


Score: 2/3 correct
❌ math_inequality: expected='112', got='We are given a convex quadrilateral $ABCD$ with the following side lengths:\n\n- $AB = 10$\n- $CD = 10$\n- $BC = 14$\n- $AD = 2\\sqrt{65}$\n\nAlso, the diagonals intersect at point $P$, and it is given that:\n\n$$\n\\text{Area}(\\triangle APB) + \\text{Area}(\\triangle CPD) = \\text{Area}(\\triangle BPC) + \\text{Area}(\\triangle APD)\n$$\n\nThis implies that the **sum of the areas of opposite triangles formed by the diagonals is equal**. This is a key geometric property that tells us that the diagonals **bisect each other in area**, i.e., the diagonals divide the quadrilateral into four triangles of equal area.\n\nSo, the total area of the quadrilateral is:\n\n$$\n\\text{Area}(ABCD) = 2 \\times \\left( \\text{Area}(\\triangle APB) + \\text{Area}(\\triangle CPD) \\right)\n$$\n\nBut we are not given the actual areas of the triangles, so we need to find the area of the quadrilateral using the given side lengths.\n\n---\n\n### Ste

In [6]:
def self_evaluate(question, prediction, expected_answer, model=MODEL):
    """
    Use the model itself as a strict grader.
    Returns True if the model says the prediction matches the expected answer; else False.
    Falls back to a simple normalized string compare if the model's reply is malformed.
    """
    import re

    system = "You are a strict grader. Reply with exactly True or False. No punctuation. No explanation."
    prompt = f"""You are grading a question-answer pair.

Return exactly True if the PREDICTION would be accepted as correct for the EXPECTED_ANSWER.
Otherwise, return False.

QUESTION:
{question}

PREDICTION:
{prediction}

EXPECTED_ANSWER:
{expected_answer}

Answer with exactly: True or False
"""

    r = call_model_chat_completions(
        prompt,
        system=system,
        model=model,
        temperature=0.0,
    )

    reply = (r.get("text") or "").strip().lower()
    if reply.startswith("true"):
        return True
    if reply.startswith("false"):
        return False

    # Fallback: simple normalization-based equality
    norm = lambda s: re.sub(r"\s+", " ", (s or "").strip().lower())
    return norm(prediction) == norm(expected_answer)


In [9]:
def self_evaluate_tests(tests, model=MODEL, grader_model=None, sleep_sec=0.2, verbose=True):
    """
    Run the tests by querying the model for each prompt, then use LLM-as-a-judge
    (self_evaluate) to determine correctness.

    Args:
        tests: list of dicts with keys: id, prompt, expected (and optionally type)
        model: model used to generate predictions
        grader_model: model used to judge correctness (defaults to `model` if None)
        sleep_sec: small delay between calls to be polite to the API
        verbose: if True, print a summary line per test

    Returns:
        rows: list of dicts with fields:
              id, expected, got, correct, status, error
    """
    import time

    judge_model = grader_model or model
    rows = []

    for t in tests:
        # 1) Get model prediction
        r = call_model_chat_completions(
            t["prompt"],
            system="You are a careful solver. Reply ONLY with the final answer, nothing else.",
            model=model,
            temperature=0.0,
        )
        got = (r.get("text") or "").strip()

        # 2) LLM-as-a-judge: strict True/False
        is_correct = self_evaluate(
            question=t["prompt"],
            prediction=got,
            expected_answer=t["expected"],
            model=judge_model,
        )

        row = {
            "id": t.get("id", "<unnamed>"),
            "expected": t["expected"],
            "got": got,
            "correct": bool(is_correct),
            "status": r.get("status"),
            "error": r.get("error"),
        }
        rows.append(row)

        if verbose:
            mark = "✅" if is_correct else "❌"
            print(f"{mark} {row['id']}: expected={row['expected']!r}, got={row['got']!r} (HTTP {row['status']})")
            if row["error"]:
                print("   error:", row["error"])

        if sleep_sec:
            time.sleep(sleep_sec)

    return rows

# Example:
results_llm_judge = self_evaluate_tests(tests, verbose=True, model=MODEL, grader_model=MODEL)


❌ math_inequality: expected='8', got='4' (HTTP 200)
✅ commonsense_ice: expected='stay the same', got='stay the same' (HTTP 200)
✅ logic_race: expected='second', got='second' (HTTP 200)


## Inference-Time Technique 1: Chain of Thought

In [ ]:
def chain_of_thought_technique(question, model=MODEL):
    result = call_model_chat_completions(
        f"""{question}\n\n
        1. First, break down the problem into smaller parts.
        2. Next, analyze each part step-by-step to workout the solution.
        3. Lastly, provide the final answer.
        3 sentences max""",
        system="You are a helpful problem solver who thinks step-by-step.",
        model=model,
        temperature=0.0,
    )
    reasoning = (result.get("text") or "").strip()
    extract_result = call_model_chat_completions(
        f"Extract only the final value, whether its a word, phrase, or a number, based on what the question is asking, from the following reasoning:\n\n{reasoning}",
        system="Output only the final answer, nothing else.",
        model=model,
        temperature=0.0,
    )
    answer = (extract_result["text"] or "").strip()
    return {
        "reasoning": reasoning,
        "answer": answer,
        "call_count": 2,
        "ok": result.get("ok")
    }

Testing CoT

Question: Let $ABCD$ be a convex quadrilateral with $AB = CD = 10$ , $BC = 14$ , and $AD = 2\sqrt{65}$ . Assume that the diagonals of $ABCD$ intersect at point $P$ , and that the sum of the areas of triangles $APB$ and $CPD$ equals the sum of the areas of triangles $BPC$ and $APD$ . Find the area of quadrilateral $ABCD$ .

Reasoning:
Let $ABCD$ be a convex quadrilateral with given side lengths and the condition on the areas of its triangles formed by the diagonals. Using the property that the sum of the areas of triangles $APB$ and $CPD$ equals the sum of the areas of triangles $BPC$ and $APD$, we deduce that the diagonals bisect each other proportionally. This leads to the conclusion that the quadrilateral is a parallelogram. The area of a parallelogram is base times height, and using the given side lengths and applying the Pythagorean theorem, we find the area of $ABCD$ is $280$. 

Thus, the area of quadrilateral $ABCD$ is $280$.

Extracted Answer: 280
Expected Answer: 1

## Inference-Time Technique 2: Self-Consistency

In [ ]:
from collections import Counter
def self_consistency(question, model=MODEL, n=3):
    answers = []
    for _ in range(n):
        result = call_model_chat_completions(
            f"{question}\n\nBriefly solve the problem in 1-3 sentences.",
            system="You are a concise problem solver.",
            model=model,
            temperature=0.7
        ).get("text", "")
        answers.append(result)
        time.sleep(0.1)

    normalized = [normalize_text(a) for a in answers]
    most_common = Counter(normalized).most_common(1)
    final_answer = next((ans for ans, norm in zip(answers, normalized) if norm == most_common[0][0]), None)
    final = call_model_chat_completions(
        f"Extract only the final value from this:\n{final_answer}",
        system="Output only the final answer, nothing else.",
        model=model,
        temperature=0.0
    ).get("text", "").strip()
    return {
        "final_answer": final,
        "vote_distribution": dict(Counter(normalized)),
        "call_count": n * 2
    }


Testing Self-Consistency

Question: Let $ABCD$ be a convex quadrilateral with $AB = CD = 10$ , $BC = 14$ , and $AD = 2\sqrt{65}$ . Assume that the diagonals of $ABCD$ intersect at point $P$ , and that the sum of the areas of triangles $APB$ and $CPD$ equals the sum of the areas of triangles $BPC$ and $APD$ . Find the area of quadrilateral $ABCD$ .

Extracted Answer: 140
Expected Answer: 112
LLM Calls Used: 6


## Inference Technique 3: Self-Reflection

In [ ]:
def self_reflection(question, model=MODEL, max_iterations=3):
    answer = call_model_chat_completions(
        f"{question}\nSolve this problem in 1-3 steps. 1-2 sentences per step",
        system="You are a concise and careful problem solver.",
        model=model,
        temperature=0.7
    ).get("text", "")

    for i in range(max_iterations):
        critique = call_model_chat_completions(
            f"Question: {question}\nCurrent Answer: {answer}\nIs this correct? Answer in 1-3 sentences.",
            system="You are a critical, and concise reviewer.",
            model=model,
            temperature=0.7
        ).get("text", "")

        answer = call_model_chat_completions(
            f"Question: {question}\nPrevious Answer: {answer}\nCritique: {critique}\nProvide a revised answer if necessary. 1 sentence",
            system="You are a concise and careful problem solver.",
            model=model,
            temperature=0.7
        ).get("text", "")

        time.sleep(0.1)
    final_answer = call_model_chat_completions(
        f"Extract only the final value from this:\n{answer}",
        system="Output only the final answer, nothing else.",
        model=model,
        temperature=0.0
    ).get("text", "").strip()
    return {
        "final_answer": final_answer,
        "call_count": max_iterations * 2 + 2
    }



Testing Self-Reflection

Question: Let $ABCD$ be a convex quadrilateral with $AB = CD = 10$ , $BC = 14$ , and $AD = 2\sqrt{65}$ . Assume that the diagonals of $ABCD$ intersect at point $P$ , and that the sum of the areas of triangles $APB$ and $CPD$ equals the sum of the areas of triangles $BPC$ and $APD$ . Find the area of quadrilateral $ABCD$ .

Final Answer: 140
Expected Answer: 112
Match: ✗
LLM Calls Used: 8


## Question Classifier

In [ ]:
def question_classifier(question, model=MODEL):
    prompt = f"""Classify the following question into ONE category: 
    'math' - mathematical calculations, equations, or numerical problems.
    'common_sense' - common sense reasoning.
    'logic' - logical problems or puzzles.
    QUESTION: {question}
    Reply with ONLY ONE of these: math, common_sense, logic"""
    
    result = call_model_chat_completions(
        prompt,
        system="You are a question classifier. Reply with only one of: math, common_sense, logic.",
        model=model,
        temperature=0.0,
    )

    category = (result.get("text") or "").strip().lower()

    technique_mapping = {
        'math' : 'cot',
        'common_sense' : 'self_consistency',
        'logic' : 'self_reflection'
    }

    return technique_mapping.get(category, 'self_reflection')
